In [3]:
import numpy as np
import tensorflow as tf
import opendatasets as od
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.python.keras import layers
from sklearn.preprocessing import MinMaxScaler

In [2]:
#od.download(
#    'https://www.kaggle.com/datasets/hassan06/nslkdd/data')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:Your Kaggle Key:Downloading nslkdd.zip to .\nslkdd


100%|██████████| 13.9M/13.9M [00:12<00:00, 1.15MB/s]


### Function to load the NSL-KDD dataset (replace with your preferred dataset path)


In [4]:
def load_nsl_kdd(path):
    with open(path, 'r') as f:
        data = f.readlines()
    X = []
    y = []  # Placeholder for labels (unsupervised learning, so y remains empty)
    for line in data:
        features = line.strip().split(',')
        X.append([float(x) for x in features[:-1]])
    return np.array(X), np.array(y)  # Return features and empty labels

### Load the NSL-KDD dataset (or your preferred dataset)


In [ ]:
# Hyperparameter tuning (consider adjusting based on your dataset and hardware)
epochs = 30
batch_size = 32

# Load the NSL-KDD dataset (or your preferred dataset)
X_train, _ = load_nsl_kdd('nslkdd/KDDTrain+_20Percent.txt')  # Replace path

# Normalize features (consider standardization or min-max scaling based on data distribution)
scaler = MinMaxScaler(feature_range=(0, 1))
X_train = scaler.fit_transform(X_train)

In [ ]:
# Define the autoencoder model (consider experimenting with different architectures)
def create_autoencoder():
    inputs = keras.Input(shape=(X_train.shape[1],))
    encoded = layers.Dense(32, activation='relu')(inputs)
    encoded = layers.Dense(16, activation='relu')(encoded)
    decoded = layers.Dense(32, activation='relu')(encoded)
    decoded = layers.Dense(X_train.shape[1], activation='sigmoid')(decoded)
    autoencoder = keras.Model(inputs=inputs, outputs=decoded)
    autoencoder.compile(loss='mse', optimizer='adam')
    return autoencoder

model = create_autoencoder()

# Train the autoencoder
model.fit(X_train, X_train, epochs=epochs, batch_size=batch_size, shuffle=True)

### Anomaly detection using reconstruction error thresholding

In [ ]:
def detect_anomalies(X_test, threshold):
    reconstructed = model.predict(X_test)
    reconstruction_errors = np.mean(np.power(X_test - reconstructed, 2), axis=1)
    anomaly_indices = np.where(reconstruction_errors > threshold)[0]
    return anomaly_indices, reconstruction_errors


### Evaluate model performance (consider using metrics like precision, recall, F1-score)
### ... (implementation details depend on the availability of ground truth labels)

In [ ]:
# Example usage (replace with your test data)
X_test, _ = load_nsl_kdd('path/to/NSL-KDD/kddcup.testing.gz')  # Replace path
X_test = scaler.transform(X_test)
threshold = 0.3  # Adjust threshold based on your dataset and desired sensitivity

anomaly_indices, reconstruction_errors = detect_anomalies(X_test, threshold)

print("Anomaly indices:", anomaly_indices)
print("Reconstruction errors:", reconstruction_errors)